# Linear Regression Model

### Set Up

In [323]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler 
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import cross_val_score
from statsmodels.stats.outliers_influence import variance_inflation_factor


In [324]:
# Importing test train splits

train_df = pd.read_csv("Data/train_set.csv")
test_df = pd.read_csv("Data/test_set.csv")

train_df['Log_Mileage'] = np.log(train_df['Mileage'])
test_df['Log_Mileage'] = np.log(test_df['Mileage'])






In [325]:
# Separating Features and Target

# Features
X_train = train_df.drop(columns=['Price', 'Log_Price', 'title', 'title_lower'], errors='ignore')
X_test = test_df.drop(columns=['Price', 'Log_Price', 'title', 'title_lower'], errors='ignore')

X_train = X_train.replace([np.inf, -np.inf], 0)
X_test = X_test.replace([np.inf, -np.inf], 0)

y_train = y_train.replace([np.inf, -np.inf], 0)
y_test = y_test.replace([np.inf, -np.inf], 0)
# Target
y_train = train_df['Log_Price']
y_test = test_df['Log_Price']


In [327]:
print("X_train:", X_train.shape)
print("y_train:", y_train.shape)

print("X_test:", X_test.shape)
print("y_test:", y_test.shape)


X_train: (2876, 388)
y_train: (2876,)
X_test: (720, 388)
y_test: (720,)


Using ridge regression to regularise the logistic regression. This will handle multicollinearity from the multiple encoded columns.

In [328]:
from sklearn.linear_model import Ridge

ridge = Ridge(alpha=1.0)
ridge.fit(X_train, y_train)

y_pred = ridge.predict(X_test)


In [329]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

y_test_actual = np.exp(y_test)
y_pred_actual = np.exp(y_pred)

print("RMSE (£):", np.sqrt(mean_squared_error(y_test_actual, y_pred_actual)))
print("MAE (£):", mean_absolute_error(y_test_actual, y_pred_actual))
print("R²:", r2_score(y_test_actual, y_pred_actual))


RMSE (£): 1345.8227782167423
MAE (£): 909.328846103928
R²: 0.9149286867799103


## Evaluation

Residuals are centered around zero with no ovvious curve and a strong predictive accuracy.

Howver there is a larger spread at lower predicted values 7-8 and tighter spead at higher values 9-10. This means there is heteroscedacity and thew model predict expensive cars more accurately.

## Refine Features

- Removing features with coefficients near zero.

- Combing Sparse Categories

- Checking multicolinearity

In [ ]:


X_numeric = X_train.select_dtypes(include=np.number)
vif = pd.DataFrame()
vif["feature"] = X_numeric.columns
vif["VIF"] = [variance_inflation_factor(X_numeric.values, i) for i in range(X_numeric.shape[1])]
print(vif.sort_values("VIF", ascending=False).head(30))


In [ ]:

scores = cross_val_score(model, X_train, y_train, cv=5, scoring='r2')
print("CV R²:", scores, "Mean:", scores.mean())


In [ ]:
y_pred_actual = np.exp(model.predict(X_test))
y_test_actual = np.exp(y_test)

mae_actual = np.mean(np.abs(y_test_actual - y_pred_actual))
print("MAE in actual price:", mae_actual)

## Fix Multicollinearity 

In [ ]:
drop_cols = [
    "Registration_Year",
    "Mileage(miles)",      # keep Log_Mileage instead
    "Engine_per_Seat",     # redundant
    "Is_Premium"           # redundant if brand dummies exist
]

X_train = X_train.drop(columns=[c for c in drop_cols if c in X_train.columns])
X_test = X_test.drop(columns=[c for c in drop_cols if c in X_test.columns])


In [ ]:
X_test.columns

In [ ]:
import pickle

# Save X
with open("X_train.pkl", "wb") as f:
    pickle.dump(X_train, f)

with open("X_test.pkl", "wb") as f:
    pickle.dump(X_test, f)

# Save y
with open("y_train.pkl", "wb") as f:
    pickle.dump(y_train, f)

with open("y_test.pkl", "wb") as f:
    pickle.dump(y_test, f)

print("Train/test sets saved successfully.")


In [ ]:
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

print(X_train.columns[:5])
print(y_train.head())
